In [34]:
import os
import sys
import torch
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '/repo/voice-cloning/voice_cloning/BigVGAN')))
from speaker_encoder import ecapa_tdnn
from model import unit_tts, unit_encoder, tts
from text.symbols import symbols
import params
from tqdm import tqdm

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ecapa_state_dict = torch.load('../voice_cloning/Grad-TTS/checkpts/speaker_encoder.pt')
grad_state_dict = torch.load('../voice_cloning/Grad-TTS/logs/text_encoder/grad_2_unit.pt')
ecapa = ecapa_tdnn.ECAPA_TDNN_SMALL_WTTH_PROJ()
add_blank = params.add_blank
nsymbols = len(symbols) + 1 if add_blank else len(symbols)
n_spks = params.n_spks
spk_emb_dim = params.spk_emb_dim
n_enc_channels = params.n_enc_channels
filter_channels = params.filter_channels
filter_channels_dp = params.filter_channels_dp
n_enc_layers = params.n_enc_layers
enc_kernel = params.enc_kernel
enc_dropout = params.enc_dropout
n_heads = params.n_heads
window_size = params.window_size

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

n_units = params.n_units

model = unit_tts.UnitGradTTS(n_units, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale).cuda()
    
# model.load_state_dict(torch.load('../voice_cloning/Grad-TTS/checkpts/gradtts_40.pt', map_location=lambda loc, storage: loc))

model.encoder = unit_encoder.UnitEncoder(n_units, n_feats, n_enc_channels, filter_channels,
                    filter_channels_dp, n_heads, n_enc_layers, enc_kernel,
                    enc_dropout, window_size, n_contentvec=0).cuda()
ecapa.load_state_dict(ecapa_state_dict)
ecapa.eval().cuda()
model.load_state_dict(grad_state_dict)



/home/sadjava/miniconda3/envs/unitspeech/lib/python3.8/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (256) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


<All keys matched successfully>

In [36]:
# model.spk_emb = ecapa
ecapa.requires_grad = False
model.encoder.requires_grad = False
model.to(device);

In [37]:
import torchaudio
import librosa
import IPython.display as ipd

In [38]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS/hifi-gan')))

In [40]:
from pathlib import Path
# Add project root to path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
sys.path.append("../voice_cloning/textlesslib")

from voice_cloning.textlesslib.textless.data.speech_encoder import SpeechEncoder


In [41]:
dense_model_name = "mhubert-base-vp_en_es_fr"
quantizer_name, vocab_size = "kmeans", 1000

unit_extractor = SpeechEncoder.by_name(
    dense_model_name=dense_model_name,
    quantizer_model_name=quantizer_name,
    vocab_size=vocab_size,
    deduplicate=True,
    need_f0=False
)

2025-04-10 16:13:06 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/sadjava/Documents/inno/genai/voice-cloning/notebooks
2025-04-10 16:13:06 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/annl/s2st/data/voxpopuli/mHuBERT/en_es_fr', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.layer9.km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2025-04-10 16:13:06 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_

In [42]:

_ = unit_extractor.cuda().eval()
unit_extractor.requires_grad_(False);

In [43]:

def process_unit(encoded, sampling_rate, hop_length):
    # A method that aligns units and durations (50Hz) extracted from 16kHz audio with
    # mel-spectrograms extracted from 22,050Hz audio.

    unit = encoded["units"].cpu().tolist()
    duration = encoded["durations"].cpu().tolist()

    duration = [int(i) * (sampling_rate // 50) for i in duration]

    expand_unit = []

    for u, d in zip(unit, duration):
        for _ in range(d):
            expand_unit.append(u)

    new_length = len(expand_unit) // hop_length * hop_length

    unit = torch.LongTensor(expand_unit)[:new_length].reshape(-1, hop_length).mode(1)[0].tolist()

    squeezed_unit = [unit[0]]
    squeezed_duration = [1]

    for u in unit[1:]:
        if u == squeezed_unit[-1]:
            squeezed_duration[-1] += 1
        else:
            squeezed_unit.append(u)
            squeezed_duration.append(1)

    unit = torch.LongTensor(squeezed_unit)
    duration = torch.LongTensor(squeezed_duration)

    return unit, duration

In [44]:
source_path = "/home/sadjava/Downloads/amir.wav"
wav, sr = librosa.load(source_path)
print('Source speech', sr)
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)

encoded = unit_extractor(wav)
unit, duration = process_unit(encoded, sr, 256)

resample_fn = torchaudio.transforms.Resample(sr, 16000)
spk = resample_fn(wav).cuda()
# torchaudio.save("Amir2.wav", spk, 24000)
spk = ecapa(spk)

Source speech 22050


In [45]:
torchaudio.save("Amir.wav", wav, 16000)

In [46]:
sys.path.append("../voice_cloning/Grad-TTS/hifi-gan")
from meldataset import mel_spectrogram

mel = mel_spectrogram(wav, n_fft, n_feats, sample_rate, hop_length,
                              win_length, f_min, f_max, center=False).squeeze()
        

In [47]:
model.train()
unit, unit_lengths = unit.unsqueeze(0).cuda(), torch.LongTensor([unit.shape[-1]]).cuda()
duration = duration.unsqueeze(0).cuda()
y, y_lengths = mel.unsqueeze(0).cuda(), torch.LongTensor([mel.shape[-1]]).cuda()
spk = spk.cuda()

In [48]:
optimizer = torch.optim.Adam(params=model.decoder.parameters(), lr=params.learning_rate)
iterations = 500

In [49]:
for iter in tqdm(range(iterations)):
    model.zero_grad()
    unit, unit_lengths = unit.detach(), unit_lengths.detach() 
    duration = duration.detach()
    y, y_lengths = y.detach(), y_lengths.detach() 
    spk = spk.detach()
    
    dur_loss, prior_loss, diff_loss = model.compute_loss(unit, unit_lengths, duration,
                                                            y, y_lengths,
                                                            spk=spk, out_size=params.out_size)
    loss = sum([dur_loss, prior_loss, diff_loss])
    loss.backward()
    optimizer.step()

100%|██████████| 500/500 [02:32<00:00,  3.28it/s]


In [52]:
source_path = "../text_encoder_dataset/LibriTTS/train-clean-100/19/198/19_198_000000_000002.wav"
wav, sr = librosa.load(source_path)
print('Source speech', sr)
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)
spk2 = resample_fn(wav).cuda()
spk2 = ecapa(spk2)
spk2.shape

Source speech 22050


torch.Size([1, 64])

In [53]:
torchaudio.save("woman.wav", wav, 16000)

In [54]:
encoded2 = unit_extractor(wav)
unit2, duration2 = process_unit(encoded2, sr, 256)
unit2, unit_lengths2 = unit2.unsqueeze(0).cuda(), torch.LongTensor([unit2.shape[-1]]).cuda()
duration2 = duration2.unsqueeze(0).cuda()

In [55]:
y_enc, y_dec, attn = model(unit2, unit_lengths2, duration2, spk=spk, n_timesteps=torch.tensor(50).cuda())

In [56]:
from models import Generator as HiFiGAN
from env import AttrDict
import json
with open('../voice_cloning/Grad-TTS/checkpts/hifigan-config.json') as f:
    h = AttrDict(json.load(f))
hifigan = HiFiGAN(h)
hifigan.load_state_dict(torch.load('../voice_cloning/Grad-TTS/checkpts/hifigan.pt', map_location=lambda loc, storage: loc)['generator'])
_ = hifigan.cuda().eval()
hifigan.remove_weight_norm()

Removing weight norm...


In [57]:

with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=22050))

In [25]:
torchaudio.save("Amir_woman.wav", audio.unsqueeze(0), 22050)

In [29]:
from model.text_encoder import TextEncoder
from collections import OrderedDict

grad_tts_state_dict = torch.load('../voice_cloning/Grad-TTS/checkpts/gradtts_40.pt')
text_encoder_state_dict = OrderedDict()
for key, value in grad_tts_state_dict.items():
    if "encoder." in key:
        new_key = key.replace("encoder.", "", 1)
        text_encoder_state_dict[new_key] = value

decoder = model.decoder
text_encoder = TextEncoder(nsymbols, n_feats, n_enc_channels, filter_channels,
                    filter_channels_dp, n_heads, n_enc_layers, enc_kernel,
                    enc_dropout, window_size).to(device)
text_encoder.load_state_dict(text_encoder_state_dict)

tts_model = tts.GradTTS(nsymbols, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale).to(device)
tts_model.decoder = decoder
tts_model.encoder = text_encoder
tts_model.eval();

In [30]:
from text import text_to_sequence, cmudict
from text.symbols import symbols
from utils import intersperse


text = "Hello my friend! How are you? I'm doing great! Nice to meet you! "
text_norm = text_to_sequence(text, dictionary=cmudict.CMUDict("../voice_cloning/Grad-TTS/resources/cmu_dictionary"))
if add_blank:
    text_norm = intersperse(text_norm, len(symbols))  # add a blank token, whose id number is len(symbols)
text_norm = torch.IntTensor(text_norm)
x = text_norm.unsqueeze(0)
x_lengths = torch.LongTensor([x.shape[-1]])

In [31]:
y_enc, y_dec, attn = tts_model(x.cuda(), x_lengths.cuda(), spk.cuda(), torch.tensor(50).cuda())

In [32]:
with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=22050))

In [33]:
torchaudio.save(
        "Amir_tts.wav", 
        audio.unsqueeze(0),  # Ensure shape [1, samples] for single-channel
        sample_rate=22050    # Match HiFiGAN's output sample rate
    )